In [ ]:
!pip install pyAudioAnalysis
!pip install eyed3
!pip install pydub
!pip install opensmile

     |████████████████████████████████| 41.3 MB 1.3 MB/s 
  Created wheel for pyAudioAnalysis: filename=pyAudioAnalysis-0.3.14-py3-none-any.whl size=41264389 sha256=d146cd1fb9c0bae3f31f37bd7312441ec79d68adef1147e33321bccd3c929046
  Stored in directory: /root/.cache/pip/wheels/85/b4/52/c6a5bdc8517a9bc4cc95aca9839f118d25139aee7881a8d210
Successfully built pyAudioAnalysis
     |████████████████████████████████| 245 kB 6.8 MB/s 
     |████████████████████████████████| 242 kB 57.1 MB/s 
  Attempting uninstall: coverage
    Found existing installation: coverage 3.7.1
    Uninstalling coverage-3.7.1:
      Successfully uninstalled coverage-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires coverage==3.7.1, but you have coverage 5.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incomp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import sys

In [ ]:
#Librerias
import eyed3
import pydub
from pyAudioAnalysis import ShortTermFeatures as aF
from pyAudioAnalysis import audioBasicIO as aIO
from pyAudioAnalysis import MidTermFeatures as mF 
import IPython
import opensmile

import numpy as np
import plotly.graph_objects as go
import plotly
import sklearn
import pandas as pd
import seaborn as sns
import librosa
import librosa.display
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from statsmodels.formula.api import logit

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    
/usr/local/lib/python3.7/dist-packages/pyAudioAnalysis/../statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# **CARGA DE DATOS**

Cargamos la ruta donde se encuentra nuestras imagenes enticketadas por el nombre

In [ ]:
Ravdess = "/content/drive/MyDrive/SinRuidoGeneral_audio_speech_actors_01-24/"


In [ ]:
ravdess=os.listdir(Ravdess)

emotion=[]
path=[]
gender=[]
intensity=[]
for i in ravdess:
    actor=os.listdir(Ravdess+i)
    for f in actor:
        fsplit= f.split('.')[0]
        fsplit=fsplit.split('-')
        emotion.append(int(fsplit[2]))
        gend=int(fsplit[6])
        if gend%2==0:
            gender.append('Female')
        else:
            gender.append('Male')
        intensity.append(int(fsplit[3]))
        for sp in emotion:
            if sp=="1":
                intensity.append('Normal')
        path.append(Ravdess + i + '/' + f)
edf=pd.DataFrame(emotion, columns=['Emotions'])
pdf=pd.DataFrame(path, columns=['Path'])
gdf=pd.DataFrame(gender, columns=['Gender'])
idf=pd.DataFrame(intensity, columns=['Intensity'])

ravdess_df=pd.concat([edf, pdf, gdf, idf], axis=1)

ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
ravdess_df.Intensity.replace({1:'Normal', 2:'Strong'}, inplace=True)
ravdess_df.head()
ravdess_df.isnull()
ravdess_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1316 entries, 0 to 1315
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Emotions   1316 non-null   object
 1   Path       1316 non-null   object
 2   Gender     1316 non-null   object
 3   Intensity  1316 non-null   object
dtypes: object(4)
memory usage: 41.2+ KB


In [ ]:
ravdess_df

,Emotions,Path,Gender,Intensity
0,neutral,/content/drive/MyDrive/SinRuidoGeneral_audio_s...,Male,Normal
1,neutral,/content/drive/MyDrive/SinRuidoGeneral_audio_s...,Male,Normal
2,neutral,/content/drive/MyDrive/SinRuidoGeneral_audio_s...,Male,Normal
3,sad,/content/drive/MyDrive/SinRuidoGeneral_audio_s...,Male,Normal
4,happy,/content/drive/MyDrive/SinRuidoGeneral_audio_s...,Male,Strong
...,...,...,...,...
1311,angry,/content/drive/MyDrive/SinRuidoGeneral_audio_s...,Female,Normal
1312,surprise,/content/drive/MyDrive/SinRuidoGeneral_audio_s...,Female,Normal
1313,sad,/content/drive/MyDrive/SinRuidoGeneral_audio_s...,Female,Strong
1314,surprise,/content/drive/MyDrive/SinRuidoGeneral_audio_s...,Female,Strong


# **Funcion para extraer las caracteristicas del audio**

In [ ]:
import audiofile
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

In [ ]:
def ExtraerCaracteristicas ( df ): #ingresa df:Dataframe con los path de las rutas de audio , sale dataAudio con las caracteristicas LowLevelDescriptors
  dataAudio = pd.DataFrame(columns =['Loudness_sma3',
 'alphaRatio_sma3',
 'hammarbergIndex_sma3',
 'slope0-500_sma3',
 'slope500-1500_sma3',
 'spectralFlux_sma3',
 'mfcc1_sma3',
 'mfcc2_sma3',
 'mfcc3_sma3',
 'mfcc4_sma3',
 'F0semitoneFrom27.5Hz_sma3nz',
 'jitterLocal_sma3nz',
 'shimmerLocaldB_sma3nz',
 'HNRdBACF_sma3nz',
 'logRelF0-H1-H2_sma3nz',
 'logRelF0-H1-A3_sma3nz',
 'F1frequency_sma3nz',
 'F1bandwidth_sma3nz',
 'F1amplitudeLogRelF0_sma3nz',
 'F2frequency_sma3nz',
 'F2bandwidth_sma3nz',
 'F2amplitudeLogRelF0_sma3nz',
 'F3frequency_sma3nz',
 'F3bandwidth_sma3nz',
 'F3amplitudeLogRelF0_sma3nz'], index = range(ravdess_df.shape[0]))


  #for n in range(ravdess_df.shape[0]):
  for n in range(ravdess_df.shape[0]):
    try:
      signal, sampling_rate = audiofile.read(df.Path[n] ,  always_2d=True)
      fn= smile.process_signal(signal, sampling_rate)
      df_reset=fn.reset_index(drop=True)


      dataAudio.iloc[n] = df_reset.mean()


    except ValueError:
      print(n)
    
  return dataAudio


In [ ]:
dato_audio = ExtraerCaracteristicas(ravdess_df)
dato_audio

,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,spectralFlux_sma3,mfcc1_sma3,mfcc2_sma3,mfcc3_sma3,mfcc4_sma3,...,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2bandwidth_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3bandwidth_sma3nz,F3amplitudeLogRelF0_sma3nz
0,0.208277,-9.781596,18.710564,0.077791,-0.015413,0.051413,32.069534,-7.132672,18.87603,-8.14896,...,17.159887,505.181152,1221.115112,-9.246956,1521.786011,849.259644,-17.152798,2528.575195,789.485962,-22.931276
1,0.162658,-8.496686,19.615801,0.040353,-0.016245,0.034139,32.636959,-6.884202,15.464112,-6.865978,...,14.644492,398.891388,1219.438354,-35.657406,1334.224976,883.158875,-42.99575,2360.031494,780.104919,-48.829475
2,0.165993,-11.29009,22.510229,0.067876,-0.016706,0.039253,35.3834,-2.617045,14.072393,1.542517,...,19.941992,427.808838,1246.579468,-1.062004,1411.583374,899.063293,-12.56007,2415.838135,970.919556,-20.037109
3,0.086215,-14.782555,21.634655,0.057953,-0.021206,0.019115,26.631346,12.154876,19.991606,-4.533904,...,17.593012,577.71582,1138.358887,-70.292297,1563.416138,845.187561,-79.632034,2624.351318,724.548523,-84.414688
4,0.223345,-14.280215,24.093435,0.095478,-0.020404,0.070683,25.445351,5.7384,-0.174057,-13.612164,...,21.288013,469.647614,1232.394165,-22.409655,1486.22644,879.689697,-36.176617,2527.62207,691.989319,-40.9118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,0.529856,-7.099772,17.708549,0.125606,-0.018213,0.200997,22.102169,-13.491487,-0.373377,-19.968569,...,15.492002,425.42334,1142.175537,-0.634229,1451.822876,732.96936,-8.248952,2527.938232,742.883606,-12.701891
1312,0.400836,-10.590408,22.426386,0.139482,-0.015654,0.198347,25.21377,-9.969764,-10.906992,-13.366549,...,15.954526,625.740417,1283.320557,-23.058954,1649.670288,936.987488,-29.377445,2619.500732,862.694275,-35.124298
1313,0.246367,-12.968383,23.461218,0.119368,-0.017483,0.062743,28.973328,1.034013,-0.912774,-1.737083,...,20.102715,425.026428,1295.382446,-12.567072,1413.421021,922.10614,-24.512669,2447.197266,943.716858,-31.402912
1314,0.738471,-6.749014,17.562441,0.143416,-0.004538,0.338085,14.911511,-20.57884,-26.357567,-19.813602,...,16.029215,654.741638,1301.253784,-14.319451,1619.960938,1109.040894,-21.318594,2634.494629,753.948059,-27.254515


validamos si no hay vacios

In [ ]:
print(dato_audio.isnull().sum())

Loudness_sma3                  0
alphaRatio_sma3                0
hammarbergIndex_sma3           0
slope0-500_sma3                0
slope500-1500_sma3             0
spectralFlux_sma3              0
mfcc1_sma3                     0
mfcc2_sma3                     0
mfcc3_sma3                     0
mfcc4_sma3                     0
F0semitoneFrom27.5Hz_sma3nz    0
jitterLocal_sma3nz             0
shimmerLocaldB_sma3nz          0
HNRdBACF_sma3nz                0
logRelF0-H1-H2_sma3nz          0
logRelF0-H1-A3_sma3nz          0
F1frequency_sma3nz             0
F1bandwidth_sma3nz             0
F1amplitudeLogRelF0_sma3nz     0
F2frequency_sma3nz             0
F2bandwidth_sma3nz             0
F2amplitudeLogRelF0_sma3nz     0
F3frequency_sma3nz             0
F3bandwidth_sma3nz             0
F3amplitudeLogRelF0_sma3nz     0
dtype: int64


# Agregramos la columna de emociones

In [ ]:
dato_audio["Emociones"] = ravdess_df.Emotions


In [ ]:
dato_audio

,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,spectralFlux_sma3,mfcc1_sma3,mfcc2_sma3,mfcc3_sma3,mfcc4_sma3,...,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2bandwidth_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3bandwidth_sma3nz,F3amplitudeLogRelF0_sma3nz,Emociones
0,0.208277,-9.781596,18.710564,0.077791,-0.015413,0.051413,32.069534,-7.132672,18.87603,-8.14896,...,505.181152,1221.115112,-9.246956,1521.786011,849.259644,-17.152798,2528.575195,789.485962,-22.931276,neutral
1,0.162658,-8.496686,19.615801,0.040353,-0.016245,0.034139,32.636959,-6.884202,15.464112,-6.865978,...,398.891388,1219.438354,-35.657406,1334.224976,883.158875,-42.99575,2360.031494,780.104919,-48.829475,neutral
2,0.165993,-11.29009,22.510229,0.067876,-0.016706,0.039253,35.3834,-2.617045,14.072393,1.542517,...,427.808838,1246.579468,-1.062004,1411.583374,899.063293,-12.56007,2415.838135,970.919556,-20.037109,neutral
3,0.086215,-14.782555,21.634655,0.057953,-0.021206,0.019115,26.631346,12.154876,19.991606,-4.533904,...,577.71582,1138.358887,-70.292297,1563.416138,845.187561,-79.632034,2624.351318,724.548523,-84.414688,sad
4,0.223345,-14.280215,24.093435,0.095478,-0.020404,0.070683,25.445351,5.7384,-0.174057,-13.612164,...,469.647614,1232.394165,-22.409655,1486.22644,879.689697,-36.176617,2527.62207,691.989319,-40.9118,happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,0.529856,-7.099772,17.708549,0.125606,-0.018213,0.200997,22.102169,-13.491487,-0.373377,-19.968569,...,425.42334,1142.175537,-0.634229,1451.822876,732.96936,-8.248952,2527.938232,742.883606,-12.701891,angry
1312,0.400836,-10.590408,22.426386,0.139482,-0.015654,0.198347,25.21377,-9.969764,-10.906992,-13.366549,...,625.740417,1283.320557,-23.058954,1649.670288,936.987488,-29.377445,2619.500732,862.694275,-35.124298,surprise
1313,0.246367,-12.968383,23.461218,0.119368,-0.017483,0.062743,28.973328,1.034013,-0.912774,-1.737083,...,425.026428,1295.382446,-12.567072,1413.421021,922.10614,-24.512669,2447.197266,943.716858,-31.402912,sad
1314,0.738471,-6.749014,17.562441,0.143416,-0.004538,0.338085,14.911511,-20.57884,-26.357567,-19.813602,...,654.741638,1301.253784,-14.319451,1619.960938,1109.040894,-21.318594,2634.494629,753.948059,-27.254515,surprise


# Exportamos la data en formato EXCEL

In [ ]:
dato_audio.to_excel('SINRUIDOGeneraldato_audio_LDD.xlsx')

# =============================FIN========================================